In [3]:
!pip install kafka-python

  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)


In [2]:
from kafka import KafkaConsumer, KafkaProducer
import json


def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')

def connect_kafka_producer(server):
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[server], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
    finally:
        return _producer

In [3]:
server = 'broker:9093'
topic_name = "first_topic"

In [5]:
from kafka.admin import KafkaAdminClient, NewTopic

admin_client = KafkaAdminClient(
    bootstrap_servers=server, 
    client_id='Franz'
)

# Create Topic
topic_list = [NewTopic(name=topic_name, num_partitions=3, replication_factor=1)]

admin_client.create_topics(new_topics=topic_list, validate_only=False)

# Remove Topic
# admin_client.delete_topics(['first_topic'])

CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='first_topic', error_code=0, error_message=None)])

In [6]:
kafka_producer = connect_kafka_producer(server)

messages = [{"character": "Iron Man", "is_avenger": "1", "universe": "MCU"},
            {"character": "Harry Potter", "universe": "J.K.Rowling", "is_villain": "0"},
            {"character": "Legolas", "universe": "J.R.R.Tolkien", "is_villain": "0", "weapon": "Mirkwood bow"}]

for i in range(len(messages)):
    publish_message(kafka_producer, topic_name, str(i), json.dumps(messages[i]))

Message published successfully.
Message published successfully.
Message published successfully.


In [7]:
consumer = KafkaConsumer(topic_name, 
                         auto_offset_reset='earliest',
                         bootstrap_servers=[server])

for msg in consumer:
    print(msg.key.decode("utf-8"), msg.value)

0 b'{"character": "Iron Man", "is_avenger": "1", "universe": "MCU"}'
2 b'{"character": "Legolas", "universe": "J.R.R.Tolkien", "is_villain": "0", "weapon": "Mirkwood bow"}'
1 b'{"character": "Harry Potter", "universe": "J.K.Rowling", "is_villain": "0"}'


KeyboardInterrupt: 

In [26]:
consumer.partitions_for_topic('first_topic')

{0, 1, 2}